### Table of Contents

* [Chapter 1: Gather data](#chapter1)
    * [Section 1.1: The WeRankDogs Twitter archive](#section_1_1)
    * [Section 1.2: The tweet imgage predictions](#section_1_2)
    * [Section 1.3:The Twitter API data](#section_1_3)
* [Chapter 2](#chapter2)
    * [Section 2.1](#section_2_1)
    * [Section 2.2](#section_2_2)
        * [Section 1.2.1](#section_1_2_1)
        * [Section 1.2.2](#section_1_2_2)
        * [Section 1.2.3](#section_1_2_3)

### Imports

In [18]:
#Importing all required packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import requests
import os
import tweepy
import json
import random
import time

# Chapter 1. Gather Data<a class="anchor" id="chapter1"></a>

## Section 1.1. The WeRateDogs Twitter archive <a class="anchor" id="section_1_1"></a>

The WeRateDogs Twitter archive. This file is provided by Udacity and downloaded manually by clicking on the provided link.

In [6]:
#Reading the WeRateDogs Twitter data from a csv file into a pandas dataframe
df_twitter_archive = pd.read_csv('twitter-archive-enhanced-2.csv', sep=',')
df_twitter_archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336000,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343000,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084000,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858000,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688000,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


## Section 1.2. The tweet image predictions <a class="anchor" id="section_1_2"></a>

The tweet image predictions, i.e., what breed of dog (or other object, animal, etc.) is present in each tweet according to a neural network. This file (image_predictions.tsv) is hosted on Udacity's servers and should be downloaded programmatically using the Requests library and the following URL: https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv

In [8]:
#Downloading the tweet image predictions programmatically using the provided URL
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
response

<Response [200]>

In [13]:
#Writing the downloaded response into a pandas df
with open('./image-predictions-3.tsv', 'wb') as file:
    file.write(response.content)
df_image_predictions = pd.read_csv('./image-predictions-3.tsv', sep='\t')
df_image_predictions.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


## Section 1.3. The Twitter API data <a class="anchor" id="section_1_3"></a>

Each tweet's retweet count and favorite ("like") count at minimum, and any additional data you find interesting. Using the tweet IDs in the WeRateDogs Twitter archive, query the Twitter API for each tweet's JSON data using Python's Tweepy library and store each tweet's entire set of JSON data in a file called tweet_json.txt file. Each tweet's JSON data should be written to its own line. Then read this .txt file line by line into a pandas DataFrame with (at minimum) tweet ID, retweet count, and favorite count. Note: do not include your Twitter API keys, secrets, and tokens in your project submission.

In [29]:
CONSUMER_KEY = 'q0GYYcW2omAPxLklz0UEcvZqG'
CONSUMER_SECRET = 'NbQZEvosoCnBP2591NlZvmQ4a0efLITQz9hvcA7eMlWjzg2jar'
OAUTH_TOKEN ='2910438674-COfGYyKSv28Tovu97sHh9iSkxE3aIbm6SWF8RKR'
OAUTH_TOKEN_SECRET = 'H2L2jDrTPbixbON2BdLCGtrGqpnkbAczYYttK3tRkIbvq'

In [30]:
#Using tweepy library, we set up Twitter API object and set rate limit parameters

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [41]:
# list of tweet ids from enhanced data set
df_tweet_ids = ['826806830735228928']

In [32]:
df_tweet_ids

0    892420643555336000
1    892177421306343000
2    891815181378084000
3    891689557279858000
4    891327558926688000
Name: tweet_id, dtype: int64

In [35]:
for status in api.user_timeline():
    print (status.id)

1220159113797062657
1220159080666296320
847480101893799936
826806830735228928
826396255257522176
826388389909950464
826387730259124224
779342963466010624


In [42]:
# creating a list for tweets with exceptions/errors 
tweets_error_list = []

# List of tweets
df_tweets = []

# start time of execution
start = time.time()

# For loop which will add each available tweet json to df_list
for tweet_id in df_tweet_ids:
    try:
        tweet = api.get_status(tweet_id, tweet_mode= 'extended')._json 
        
        favorites = tweet['favorite_count'] # number of favorites for the tweet
        retweets = tweet['retweet_count'] # number of retweets
        user_followers = tweet['user']['followers_count'] # number of followers of the user who tweeted
        user_favourites = tweet['user']['favourites_count'] # number of favourites for the user who tweeted
        date_time = tweet['created_at'] # the timestamp i.e. date and time of creation of the tweet
        
        df_tweets.append({'tweet_id': int(tweet_id),
                        'favorites': int(favorites),
                        'retweets': int(retweets),
                        'user_followers': int(user_followers),
                        'user_favourites': int(user_favourites),
                        'date_time': pd.to_datetime(date_time)})
    except Exception as e:
        print(str(tweet_id)+ " _ " + str(e))
        tweets_error_list.append(tweet_id)
        
# end time for excution
end = time.time()

#printing time for execution
print("Total time taken for execution", end - start)

Total time taken for execution 0.47919797897338867


In [43]:
df_tweets

[{'tweet_id': 826806830735228928,
  'favorites': 0,
  'retweets': 3,
  'user_followers': 323,
  'user_favourites': 10,
  'date_time': Timestamp('2017-02-01 14:58:04+0000', tz='UTC')}]

In [21]:
# Get a list of tweet_ids from the enhanced twitter archive to use for 
# downloading with the Twitter API

tweet_ids = df_twitter_archive.tweet_id.values
print("# of tweet_ids: " + str(len(tweet_ids)) + "\n")

count = 0
tweet_json_list = []
fails_dict = {}

start_time = time.time()

for tweet_id in tweet_ids:
    count += 1
    try:
        # attempt to get the tweet's JSON data and append to the tweet JSON list
        tweet = api.get_status(tweet_id, tweet_mode = 'extended')
        tweet_json_list.append(tweet._json)
    except tweepy.TweepError as err:
        # save the error to the fail dictionary for review
        print("TweepError for id:  " + str(tweet_id))
        fails_dict[tweet_id] = err
        pass
    # To save space, only print out loop/tweet id for every 100th tweet
    if count % 100 == 0:
        print("loop # " + str(count))

elapsed_time = time.time() - start_time

elapsed_time_str = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))
print("\nTime elapsed (HH:MM:SS):  " + elapsed_time_str + "\n")

# display list of tweets with errors
print("Number of TweepErrors:  {}\n".format(len(fails_dict)))
for tweet_id in fails_dict:
    print(tweet_id, fails_dict[tweet_id])

# of tweet_ids: 2356

TweepError for id:  892420643555336000
TweepError for id:  892177421306343000
TweepError for id:  891815181378084000
TweepError for id:  891689557279858000
TweepError for id:  891327558926688000
TweepError for id:  891087950875897000
TweepError for id:  890971913173991000
TweepError for id:  890729181411237000
TweepError for id:  890609185150312000
TweepError for id:  890240255349198000
TweepError for id:  890006608113172000
TweepError for id:  889880896479866000
TweepError for id:  889665388333682000
TweepError for id:  889638837579907000
TweepError for id:  889531135344209000
TweepError for id:  889278841981685000
TweepError for id:  888917238123831000
TweepError for id:  888804989199671000
TweepError for id:  888554962724278000
TweepError for id:  888202515573088000
TweepError for id:  888078434458587000
TweepError for id:  887705289381826000
TweepError for id:  887517139158093000
TweepError for id:  887473957103951000
TweepError for id:  887343217045368000
Twe

KeyboardInterrupt: 